In [1]:
import os
import pandas as pd
import time
import glob
from IPython.display import clear_output
from pyspark.sql import SparkSession
import pyspark.sql.functions as sql_f
from pyspark.sql.types import *
from pyspark.sql.functions import to_date, datediff, floor, col, avg, substring, when, length, lpad
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import unix_timestamp

spark = SparkSession.builder.getOrCreate()

In [2]:
#Function to measure execution time...
def time_execution(task_name, func):
    start_time = time.time()
    result = func()
    end_time = time.time()
    duration = end_time - start_time
    print(f"{task_name} executed in {duration:.2f} seconds")
    return result, duration


## 1) Loading the data

In [3]:
#Synthea Batch Generator with Concatenation of csv files

#Synthea Configuration 
total_patients = 1000  
batch_size = 20
state = "Massachusetts"  
age_range = "30-85"  
base_seed = 12345  
output_dir = "./output" 

#Initial setup to ensure java is installed as well as the "synthea-with-dependencies.jar" file needed to create the patient records
def setup_environment():
    """Install Java and download Synthea if not already present"""
    if not os.path.exists("/usr/bin/java"):
        print("Installing Java...")
        !sudo apt-get update -qq > /dev/null
        !sudo apt-get install -y openjdk-11-jdk-headless > /dev/null
        clear_output()
        print("Java installed")
    
    if not os.path.exists("synthea-with-dependencies.jar"):
        print("Downloading Synthea...")
        !wget -q https://github.com/synthetichealth/synthea/releases/download/master-branch-latest/synthea-with-dependencies.jar
        clear_output()
        print("Synthea downloaded")
    
    os.makedirs(output_dir, exist_ok=True)

#Batch generation of patients to avoid errors due to kernel reconnecting...
def generate_batch(batch_num, patients_in_batch, current_seed):
    """Generate one batch of synthetic patients"""
    try:
        print(f"Batch {batch_num}: Generating {patients_in_batch} patients (seed: {current_seed})")
        
        !java -jar synthea-with-dependencies.jar \
          -p {patients_in_batch} \
          -s {current_seed} \
          -a "{age_range}" \
          --exporter.baseDirectory "{output_dir}" \
          --exporter.fhir.export=False \
          --exporter.csv.export=True \
          --exporter.csv.folder_per_run=true \
          {state}
        
        return True
    except Exception as e:
        print(f"Error in batch {batch_num}: {str(e)}")
        return False

#Combining csv files based on their filenames
def concatenate_all_csvs():
    """Combine all generated CSV files by their type"""
    csv_files = glob.glob(f"{output_dir}/**/*.csv", recursive=True)
    
    if not csv_files:
        print("No CSV files found to concatenate")
        return None
    
    combined_data = {}
    
    for filepath in csv_files:
        filename = os.path.basename(filepath)
        file_type = filename.split('.')[0]
        
        try:
            df = pd.read_csv(filepath)
            
            if file_type in combined_data:
                combined_data[file_type] = pd.concat([combined_data[file_type], df], ignore_index=True)
            else:
                combined_data[file_type] = df
                
        except Exception as e:
            print(f"Could not process {filename}: {str(e)}")
    
    #Saving combined files
    combined_dir = f"{output_dir}/combined"
    os.makedirs(combined_dir, exist_ok=True)
    
    for file_type, df in combined_data.items():
        output_path = f"{combined_dir}/{file_type}.csv"
        df.to_csv(output_path, index=False)
        print(f"Saved {len(df)} records to {output_path}")
    
    return combined_data

#Calling functions from above:
if __name__ == "__main__":
    setup_environment()

    start_time = time.time()
    completed = 0
    batch_num = 1
    
    print(f"Starting generation of {total_patients} patients in batches of {batch_size}...")
    
    while completed < total_patients:
        current_batch_size = min(batch_size, total_patients - completed)
        current_seed = base_seed + completed
        
        #Retrying logic (3 attempts per batch)
        success = False
        for attempt in range(3):
            if generate_batch(batch_num, current_batch_size, current_seed):
                success = True
                break
            time.sleep(5)
        
        if success:
            completed += current_batch_size
            progress = completed / total_patients * 100
            print(f"Progress: {completed}/{total_patients} ({progress:.1f}%)")
            batch_num += 1
        else:
            print(f"Failed batch {batch_num} after 3 attempts")
            break
    
    #Concatenate results
    print("\nCombining all CSV files...")
    combined_data = concatenate_all_csvs()

    
    elapsed = (time.time() - start_time) / 60
    print(f"\n{'='*40}")
    print(f"COMPLETED IN {elapsed:.1f} MINUTES")
    print(f"Total patients generated: {completed}/{total_patients}")
    
    if combined_data:
        print("\nCOMBINED FILES SUMMARY:")
        for file_type, df in combined_data.items():
            print(f"- {file_type}.csv: {len(df)} records")
    else:
        print("\nNo files were combined")

Starting generation of 1000 patients in batches of 20...
Batch 1: Generating 20 patients (seed: 12345)
SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#noProviders for further details.
Scanned 88 modules and 152 submodules.
Loading submodule modules/allergies/allergy_panel.json
Loading submodule modules/allergies/drug_allergy_incidence.json
Loading submodule modules/allergies/environmental_allergy_incidence.json
Loading submodule modules/allergies/food_allergy_incidence.json
Loading submodule modules/allergies/immunotherapy.json
Loading submodule modules/allergies/outgrow_env_allergies.json
Loading submodule modules/allergies/outgrow_food_allergies.json
Loading submodule modules/allergies/severe_allergic_reaction.json
Loading submodule modules/anemia/anemia_sub.json
Loading submodule modules/breast_cancer/chemotherapy_breast.json
Loading submodule modules/breast_cancer/hormone_diagnosis.json
L

Loading Lookup Table: ace_arb_lisinopril_product_distribution.csv
Loading Lookup Table: ace_arb_losartan_product_distribution.csv
Loading Lookup Table: ace_arb_quinapril_product_distribution.csv
Loading Lookup Table: ace_arb_ramipril_product_distribution.csv
Loading Lookup Table: ace_arb_sacubitril_valsartan_product_distribution.csv
Loading Lookup Table: ace_arb_telmisartan_product_distribution.csv
Loading Lookup Table: ace_arb_valsartan_product_distribution.csv
Loading submodule modules/medications/beta_blocker.json
Loading Lookup Table: beta_blocker_ingredient_distribution.csv
Loading Lookup Table: beta_blocker_atenolol_product_distribution.csv
Loading Lookup Table: beta_blocker_bisoprolol_product_distribution.csv
Loading Lookup Table: beta_blocker_bisoprolol_hydrochlorothiazide_product_distribution.csv
Loading Lookup Table: beta_blocker_carvedilol_product_distribution.csv
Loading Lookup Table: beta_blocker_labetalol_product_distribution.csv
Loading Lookup Table: beta_blocker_metopro

Loading Lookup Table: vhd_pr.csv
Loading module modules/vhd_tricuspid.json
Loading Lookup Table: vhd_tr.csv
Loading Lookup Table: vhd_ts.csv
Loading module modules/wellness_encounters.json
Running with options:
Population: 20
Seed: 12345
Provider Seed:1746486433196
Reference Time: 1746486433196
Location: Massachusetts
Min Age: 30
Max Age: 85
2 -- Rasheeda241 Mirian768 Dach178 (35 y/o F) Boston, Massachusetts  (50927)
1 -- Lorenzo669 Urrutia540 (67 y/o M) Chicopee, Massachusetts  (145814)
3 -- Jacque955 Jin479 Stroman228 (57 y/o F) Shrewsbury, Massachusetts DECEASED (86979)
3 -- Suellen60 McCullough561 (68 y/o F) Shrewsbury, Massachusetts  (99533)
4 -- Keven605 Daugherty69 (50 y/o M) Dalton, Massachusetts  (73435)
6 -- Darrell400 Muller251 (50 y/o M) Lowell, Massachusetts  (69104)
5 -- Angela104 Bella510 Wuckert783 (81 y/o F) South Yarmouth, Massachusetts  (118352)
7 -- Claudia969 Escobedo608 (35 y/o F) Lynn, Massachusetts  (51109)
9 -- Alfred550 West559 (31 y/o M) Belmont, Massachusett

Loading submodule modules/heart/operative_status.json
Loading submodule modules/heart/or_blood.json
Loading Lookup Table: or_blood_anemia_check.csv
Loading Lookup Table: or_blood_platelet_check.csv
Loading Lookup Table: or_blood_plasma_check.csv
Loading submodule modules/heart/savrepair/operative_status.json
Loading submodule modules/heart/savreplace/operative_status.json
Loading submodule modules/heart/stemi_fibrinolytic.json
Loading submodule modules/heart/stemi_pathway.json
Loading submodule modules/heart/tavr/alt_access.json
Loading submodule modules/heart/tavr/operation.json
Loading submodule modules/heart/tavr/operative_status.json
Loading submodule modules/heart/tavr/outcomes.json
Loading submodule modules/heart/tavr/postop.json
Loading submodule modules/heart/vhd_risks.json
Loading submodule modules/hiv/art_sequence.json
Loading submodule modules/hiv/art_sequence_1987_1994.json
Loading submodule modules/hiv/art_sequence_1995_1996.json
Loading submodule modules/hiv/art_sequence_

Loading module modules/home_hospice_snf.json
Loading module modules/homelessness.json
Loading module modules/hospice_treatment.json
Loading module modules/hypertension.json
Loading module modules/hypothyroidism.json
Loading module modules/injuries.json
Loading module modules/kidney_transplant.json
Loading module modules/lung_cancer.json
Loading module modules/lupus.json
Loading module modules/mTBI.json
Loading module modules/med_rec.json
Loading module modules/mend_program.json
Loading module modules/metabolic_syndrome_care.json
Loading module modules/metabolic_syndrome_disease.json
Loading module modules/myocardial_infarction.json
Loading module modules/opioid_addiction.json
Loading module modules/osteoarthritis.json
Loading module modules/osteoporosis.json
Loading module modules/pregnancy.json
Loading module modules/prescribing_opioids_for_chronic_pain_and_treatment_of_oud.json
Loading module modules/rheumatoid_arthritis.json
Loading module modules/self_harm.json
Loading module modul

...

Could not process providers.csv: No columns to parse from file
Could not process organizations.csv: No columns to parse from file
Could not process payers.csv: No columns to parse from file
Could not process providers.csv: No columns to parse from file
Could not process organizations.csv: No columns to parse from file
Could not process payers.csv: No columns to parse from file


/tmp/ipykernel_147631/2039501488.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data[file_type] = pd.concat([combined_data[file_type], df], ignore_index=True)


Could not process providers.csv: No columns to parse from file
Could not process organizations.csv: No columns to parse from file
Could not process payers.csv: No columns to parse from file


/tmp/ipykernel_147631/2039501488.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data[file_type] = pd.concat([combined_data[file_type], df], ignore_index=True)


Could not process providers.csv: No columns to parse from file
Could not process organizations.csv: No columns to parse from file
Could not process payers.csv: No columns to parse from file
Could not process providers.csv: No columns to parse from file
Could not process organizations.csv: No columns to parse from file
Could not process payers.csv: No columns to parse from file
Could not process medications.csv: No columns to parse from file
Could not process devices.csv: No columns to parse from file
Could not process providers.csv: No columns to parse from file
Could not process imaging_studies.csv: No columns to parse from file
Could not process organizations.csv: No columns to parse from file
Could not process supplies.csv: No columns to parse from file
Could not process patients.csv: No columns to parse from file
Could not process encounters.csv: No columns to parse from file
Could not process payers.csv: No columns to parse from file
Could not process conditions.csv: No columns to

In [4]:
#moving combined csv files into HDFS
!hdfs dfs -mkdir -p /synthea_output 2> /dev/null
!hdfs dfs -put -f ./output/combined/*.csv /synthea_output/
!rm -rf ./output 2> /dev/null && echo "Local files cleaned up" || echo "Error cleaning local files"

Local files cleaned up


In [ ]:
#uncomment this cell to save files to local downloads:

# !mkdir -p ~/Downloads/synthea_csvs
# !hdfs dfs -get /synthea_output/*.csv ~/Downloads/synthea_csvs/

## 2) Creating the spark dataframes 

In [5]:
#setting path to HDFS folder
path = '/synthea_output/'


In [6]:
#Patient files
observations = spark.read.csv(path+"observations.csv", header=True)
patient = spark.read.csv(path+"patients.csv", header=True) 
#Medical files
careplans = spark.read.csv(path+"careplans.csv", header=True)
conditions = spark.read.csv(path+"conditions.csv", header=True)
procedures=spark.read.csv(path+"procedures.csv", header=True)
encounters = spark.read.csv(path+"encounters.csv", header=True)
medications = spark.read.csv(path+"medications.csv", header=True)
#Insurance and hospital files
payer_transitions=spark.read.csv(path+"payer_transitions.csv", header=True)
payers=spark.read.csv(path+"payers.csv", header=True)
providers=spark.read.csv(path+"providers.csv", header=True)
organizations=spark.read.csv(path+"organizations.csv", header=True)

## 3) Cleaning dataframes and renaming variables 

In [7]:
#Renaming columns
patient = (
    patient.withColumnRenamed("Id", "patient_id")
           .withColumnRenamed("MARITAL", "patient_marital")
           .withColumnRenamed("RACE", "patient_race")
           .withColumnRenamed("ETHNICITY", "patient_ethnicity")
           .withColumnRenamed("GENDER", "patient_gender")
           .withColumnRenamed("ZIP", "patient_zip")
)
encounters = (
    encounters.withColumnRenamed("PATIENT", "patient_id")
              .withColumnRenamed("Id", "encounter_id")
              .withColumnRenamed("DESCRIPTION", "encounter_discription")
              .withColumnRenamed("CODE", "encounter_code")
              .withColumnRenamed("START", "encounter_start")
              .withColumn("encounter_start", to_date("encounter_start"))
              .withColumnRenamed("STOP", "encounter_stop")
              .withColumn("encounter_stop", to_date("encounter_stop"))
              .withColumn("PATIENT COST", col("TOTAL_CLAIM_COST") - col("PAYER_COVERAGE"))
              .withColumnRenamed("PAYER", "payer_id")
              .withColumnRenamed("ORGANIZATION", "organization_id")
              .withColumnRenamed("PROVIDER", "provider_id")
)
careplans = (
    careplans.withColumnRenamed("PATIENT", "patient_id")
             .withColumnRenamed("Id", "careplan_id")
             .withColumnRenamed("ENCOUNTER", "encounter_id")
             .withColumnRenamed("DESCRIPTION", "careplan_descriptions")
             .withColumnRenamed("CODE", "careplan_code")
)
procedures = (
    procedures.withColumnRenamed("PATIENT", "patient_id")
              .withColumnRenamed("ENCOUNTER", "encounter_id")
              .withColumnRenamed("DESCRIPTION", "procedure_descriptions")
              .withColumnRenamed("CODE", "procedure_code")
              .withColumnRenamed("DATE", "procedure_date")
              .withColumnRenamed("BASE_COST", "procedure_cost")
)
conditions = (
    conditions.withColumnRenamed("PATIENT", "patient_id")
              .withColumnRenamed("ENCOUNTER", "encounter_id")
              .withColumnRenamed("DESCRIPTION", "condition_description")
              .withColumnRenamed("CODE", "condition_code")
              .withColumnRenamed("START", "condition_start")
              .withColumnRenamed("END", "condition_end")
)
observations = (
    observations.withColumnRenamed("PATIENT", "patient_id")
                .withColumnRenamed("ENCOUNTER", "encounter_id")
                .withColumnRenamed("DATE", "observation_date")
                .withColumn("observation_date", to_date("observation_date"))
                .withColumn("obs_value", col("VALUE").cast("double"))
                .withColumnRenamed("CODE", "observation_code")
                .withColumnRenamed("DESCRIPTION", "observation_description")
)
medications = (
    medications.withColumnRenamed("START", "medication_start")
               .withColumn("medication_start", to_date("medication_start"))
               .withColumnRenamed("STOP", "medication_stop")
               .withColumn("medication_stop", to_date("medication_stop"))
               .withColumnRenamed("PATIENT", "patient_id")
               .withColumnRenamed("PAYER", "payer_id")
               .withColumnRenamed("ENCOUNTER", "encounter_id")
               .withColumnRenamed("CODE", "medication_code")
               .withColumnRenamed("DESCRIPTION", "medication_description")
)
payer_transitions = (
    payer_transitions.withColumnRenamed("PATIENT", "patient_id")
                     .withColumnRenamed("PAYER", "payer_id")
)
payers = (
    payers.withColumnRenamed("Id", "payer_id")
          .withColumnRenamed("NAME", "payer_name")
          .withColumnRenamed("OWNERSHIP", "payer_ownership")
)
providers = (
    providers.withColumnRenamed("Id", "provider_id")
             .withColumnRenamed("SPECIALITY", "provider_specialty")
)
organizations = (
    organizations.withColumnRenamed("Id", "organization_id")
                 .withColumnRenamed("NAME", "organization_name")
                 .withColumnRenamed("ZIP", "organization_zip")
)
organizations = organizations.withColumn(
    "organization_zip",
    col("organization_zip").cast("string")
)

#adding leading 0's to zip codes to retain their information
organizations = organizations.withColumn(
    "organization_zip",
    when(length(col("organization_zip")) == 4,  
    lpad(col("organization_zip"), 5, "0")
).otherwise(col("organization_zip")))   

In [8]:
#Merge together dataframes on various id fields that will be used for ML modeling...
encounters = (
    encounters
    .join(payers.select("payer_id", "payer_name", "payer_ownership"), on="payer_id", how="left")
    .join(organizations.select("organization_id", "organization_name", "organization_zip"), on="organization_id", how="left")
    .join(providers.select("provider_id", "provider_specialty"), on="provider_id", how="left")
    .join(procedures.select("encounter_id", "procedure_descriptions", "procedure_code"), on="encounter_id", how="left")
    .join(patient.select("patient_id", "BIRTHDATE", "patient_marital", "patient_race", "patient_ethnicity", "patient_gender", "patient_zip"), on="patient_id", how="left")
    .withColumn("age_at_encounter", floor(datediff(col("encounter_start"), col("BIRTHDATE")) / 365.25))
)


In [9]:
encounters.show(5)

25/05/05 23:34:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------+--------------+--------------+---------------------+-------------------+----------------+--------------+----------+-----------------+------------+----------+---------------+--------------------+----------------+------------------+----------------------+--------------+----------+---------------+------------+-----------------+--------------+-----------+----------------+
|          patient_id|        encounter_id|         provider_id|     organization_id|            payer_id|encounter_start|encounter_stop|ENCOUNTERCLASS|encounter_code|encounter_discription|BASE_ENCOUNTER_COST|TOTAL_CLAIM_COST|PAYER_COVERAGE|REASONCODE|REASONDESCRIPTION|PATIENT COST|payer_name|payer_ownership|   organization_name|organization_zip|provider_specialty|procedure_descriptions|procedure_code| BIRTHDATE|patient_marital|patient_race|patient_ethnicity|patient_gender|patient_zip|ag

In [10]:
encounters.columns

['patient_id',
 'encounter_id',
 'provider_id',
 'organization_id',
 'payer_id',
 'encounter_start',
 'encounter_stop',
 'ENCOUNTERCLASS',
 'encounter_code',
 'encounter_discription',
 'BASE_ENCOUNTER_COST',
 'TOTAL_CLAIM_COST',
 'PAYER_COVERAGE',
 'REASONCODE',
 'REASONDESCRIPTION',
 'PATIENT COST',
 'payer_name',
 'payer_ownership',
 'organization_name',
 'organization_zip',
 'provider_specialty',
 'procedure_descriptions',
 'procedure_code',
 'BIRTHDATE',
 'patient_marital',
 'patient_race',
 'patient_ethnicity',
 'patient_gender',
 'patient_zip',
 'age_at_encounter']

In [11]:
encounters.count()

2362312